# Deep Learning Project

**Group:** Songbird  
**Members:** Charlotte de Vries, Jiazhen Tang, Paulo Zirlis

In [ ]:
# Setup block
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from transformers import TFAutoModelForImageClassification
from sklearn.utils import class_weight, resample
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Setup timer
class ConvergenceTimer(Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        print("Training started...")

    def on_train_end(self, logs={}):
        self.end_time = time.time()
        self.total_time = self.end_time - self.start_time
        print(f"\nTraining finished.")
        print(f"Time to converge: {self.total_time:.2f} seconds ({self.total_time/60:.2f} minutes)")


print("Setup OK")

In [ ]:
# Check if GPU is available
import tensorflow as tf
print(tf.config.list_physical_devices())

## 1. Project Overview

### 1.1 Objective*

Apply different deep learning architectures to the visual classification task of identifying brain tumors in MRI images and compare them based on accuracy and time to converge.


### 1.2 Neural Network Architectures

We will implement and compare the following architectures:
- Custom Convolutional Neural Network (CNN) with keras sequential
- Custom Residual Network (ResNet)
- Pre-trained Residual Network (ResNet50)
- Pre-trained Residual Network (ResNet50) with fine-tuning
- Pre-trained Visual Transformer (ViT)
- Pre-trained Visual Transformer (ViT) with fine-tuning


### 1.3 Dataset Description

The dataset includes high-resolution CT and MRI images captured from multiple patients, with each image labeled with the corresponding tumor type (e.g., glioma, meningioma, etc.). For this project we will focus solely on the **MRI** images for simplicity. The dataset's creator collected these data from different sources to assist researchers and healthcare professionals in developing AI models for the automatic detection, classification, and segmentation of brain tumors.

The images are divided as follows:
- Healty images: 2000
- Tumor images: 3000
    - Meningioma: 1112
    - Glioma: 672
    - Pituitary: 629
    - Tumor: 587
- **Total of images:** 5000

Source: [Brain tumor multimodal image (Kaggle)](https://www.kaggle.com/datasets/murtozalikhon/brain-tumor-multimodal-image-ct-and-mri/data)

***

## 2. Data Preprocessing

### 2.1 Load data

In [ ]:
# Load data

# 1. SETUP PATHS
dataset_path = 'Data/Brain Tumor MRI images'

print(f"Checking contents of: {dataset_path}")
try:
    items = os.listdir(dataset_path)
    print("Found these items:", items)
except:
    print("Error: The dataset_path does not exist.")

filepaths = []
labels = []



# Get list of all folders in the main directory
all_items = os.listdir(dataset_path)

for item in all_items:
    item_path = os.path.join(dataset_path, item)
    
    # We only care if it's a folder (directory)
    if os.path.isdir(item_path):
        
        # --- CASE A: The 'Healthy' Folder ---
        if 'healthy' in item.lower():
            print(f"Processing Healthy folder: {item}")
            for filename in os.listdir(item_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
                    filepaths.append(os.path.join(item_path, filename))
                    labels.append('Healthy')
                    
        # --- CASE B: The 'Tumour' Folder (Anything that isn't Healthy) ---
        else:
            print(f"Processing Tumour folder: {item}")
            for filename in os.listdir(item_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
                    full_path = os.path.join(item_path, filename)
                    name_lower = filename.lower()
                    
                    # Determine Subtype based on filename
                    if 'glioma' in name_lower:
                        label = 'Glioma'
                    elif 'meningioma' in name_lower:
                        label = 'Meningioma'
                    elif 'pituitary' in name_lower:
                        label = 'Pituitary'
                    else:
                        label = 'Tumor (Unspecified)' 
                    
                    filepaths.append(full_path)
                    labels.append(label)

# Create DataFrame
df = pd.DataFrame({'filepath': filepaths, 'label': labels})

# Check results
print(f"Total images found: {len(df)}")
print(df['label'].value_counts())


### 2.2 Oversampling

In [ ]:
# 1. Identify the majority class count
max_count = df['label'].value_counts().max()
print(f"Target count per class: {max_count}")

# 2. Separate the dataframe by class
groups = df.groupby('label')

# 3. Create a list to hold the balanced dataframes
balanced_dfs = []

for label, group_df in groups:
    # If this group is smaller than the max, oversample it
    if len(group_df) < max_count:
        
        # Resample logic:
        # replace=True: This allows duplication (essential for oversampling)
        # n_samples=max_count: Target number of samples
        oversampled_group = resample(group_df, 
                                     replace=True, 
                                     n_samples=max_count, 
                                     random_state=42)
        balanced_dfs.append(oversampled_group)
        print(f"Oversampled {label} from {len(group_df)} to {len(oversampled_group)}")
        
    else:
        # If it's the majority class (Healthy), just keep it as is
        balanced_dfs.append(group_df)
        print(f"Kept {label} at {len(group_df)}")

# 4. Concatenate all back into one DataFrame
df_balanced = pd.concat(balanced_dfs).reset_index(drop=True)

# 5. Shuffle the dataset so classes aren't clustered together
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# 6. Verify the new counts
print("\nNew Label Distribution:")
print(df_balanced['label'].value_counts())

# 7. Update your main df variable
df = df_balanced

### 2.3 Train-test split

In [ ]:
# 2. Split: 80% Train, 20% Test (using stratify to keep classes balanced)
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42, stratify=df['label'])

# 3. Split Train again to get Validation set (e.g. 10% of total)
train_df, val_df = train_test_split(train_df, test_size=0.125, shuffle=True, random_state=42, stratify=train_df['label'])

print(f"Train size: {len(train_df)}")
print(f"Val size:   {len(val_df)}")
print(f"Test size:  {len(test_df)}")

# 4. Visualize to confirm labels are correct

plt.figure(figsize=(14, 8))

# Get a random sample to check
sample_df = df.sample(10)

for i, (index, row) in enumerate(sample_df.iterrows()):
    plt.subplot(2, 5, i + 1)
    img = mpimg.imread(row['filepath'])
    plt.imshow(img, cmap='gray')
    plt.title(f"{row['label']}\n{os.path.basename(row['filepath'])[:10]}...", fontsize=9) # Show label + part of filename
    plt.axis('off')

plt.tight_layout()
plt.show()

### 2.4 Build Keras generators

In [ ]:
### Images to correct format

# 1. Define image size and batch size
IMG_SIZE = (256, 256) 
BATCH_SIZE = 32

# 2. Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

# 3. Build generators FROM DATAFRAMES
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

***

## 3. Custom Convolutional Neural Network

The Convolutional Neural Network (CNN) model designed for this project consists of four convolutional blocks followed by a final block with pooling, dropout and fully connected layers. Each convolutional block has a convolution layer, batch normalization, activation function (ReLU) and max pooling. Early stopping was added to control for overfitting and underfitting. Batch normalization was used to improve training speed and stability. Dropout was included in the final block to further prevent overfitting. The model was compiled with the Adam optimizer, categorical cross-entropy loss function, and accuracy as the evaluation metric.

<br>

### 3.1 CNN Architecture

**Input and Data Augmentation**
- Input layer: shape (256, 256, 1)
- Data Augmentation: Random rotations and horizontal flips

**First Convolutional Block**
- Conv2d layer: 32 filters, 3x3 kernel, stride of 1, same padding
- Batch Normalization
- ReLU Activation
- MaxPooling2d layer: 2x2 pool size, stride of 2.

**Other Convolutional Blocks**
- same as the first block but with increasing number of filters (64, 128, 256)

**Classifier Head**
- Global Average Pooling layer
- Dense layer: 64 units, ReLU activation
- Dropout layer: 0.3 dropout rate
- Dense layer: 5 units (nº of classes), Softmax activation

In [ ]:
### CNN Architecture

# Seed for reproducibility
np.random.seed(42)

# Custom CNN
CNN = keras.Sequential([
    
    # Input
    layers.InputLayer(shape=[256, 256, 1]),
    
    # Data Augmentation
    layers.RandomFlip("horizontal"), # flip images horizontally
    layers.RandomRotation(0.1),      # rotate images randomly by 10%


    # 1st Convolutional Block
    layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # 2nd Convolutional Block
    layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # 3rd Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # 4th Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2, strides=2),

    # 5th Convolutional Block
    layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.SpatialDropout2D(0.2),
    layers.MaxPool2D(pool_size=2, strides=2),

    # 6th Convolutional Block
    layers.Conv2D(filters=512, kernel_size=3, strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.SpatialDropout2D(0.2),
    layers.MaxPool2D(pool_size=2, strides=2),

    # Classifier Head
    layers.GlobalAveragePooling2D(),
    layers.Dense(units=128),
    layers.Dense(units=64),
    layers.Dense(units=5, activation='softmax')  # 5 classes
])

CNN.summary()


# Compile the model
CNN.compile(
    optimizer = Adam(0.001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

### 3.2 Train

In [ ]:
### ============== Callbacks ==============

# Early stopping
early_stopping = EarlyStopping(
    min_delta = 0.001,
    patience = 15,
    restore_best_weights = True
)

# Reduce LR on plateau
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,       # Reduce LR by 80% when stuck
    patience=5,       # Wait 5 epochs before reducing
    min_lr=1e-6,      
    verbose=1
)

# Checkpoint
checkpoint = ModelCheckpoint(
    filepath='best_CNN.keras',    # Naming the file
    monitor='val_accuracy',       # What to monitor
    mode='max',                   # 'max' for accuracy, 'min' for loss
    save_best_only=True,          
    verbose=0                     # Print a message when saving
)

# Timer
callback_timer = ConvergenceTimer()



### ============== Training ==============

# Fit the model
hist_CNN = CNN.fit(
    train_gen,
    validation_data = val_gen,
    batch_size = 32,
    epochs = 50,
    callbacks = [early_stopping, lr_scheduler, checkpoint, callback_timer],
    verbose = 1
)

print("Training complete!")

### 3.3 Evaluate

In [ ]:
### ============== Evaluation ==============

# Loss
plt.plot(hist_CNN.history['loss'], label='train_loss')
plt.plot(hist_CNN.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Accuracy
plt.plot(hist_CNN.history['accuracy'], label='train_accuracy')
plt.plot(hist_CNN.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

***

## 4. Pre-trained Residual Network
Charlotte

### 4.1 ResNet **without finetuning**

#### 4.1.1 Preprocessing data

In [ ]:
ds_train_c5_, ds_val_c5_ = image_dataset_from_directory(
    '/tmp/BrainTumorDataset',
    validation_split=0.2,
    subset='both',
    seed=42,
    image_size=(224,224),
    batch_size=32,
    label_mode='categorical'
)

#### 4.1.2 Data preprocessing

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input

def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

ds_train_c5 = ds_train_c5_.map(preprocess)
ds_val_c5 = ds_val_c5_.map(preprocess)

#### 4.1.3 Data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

def augment(image, label):
    image = data_augmentation(image)
    return image, label

ds_train_c5 = ds_train_c5_.map(augment).map(preprocess)

#### 4.1.4 Create the model

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

## pretrained base
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False   # Freeze weights

## attach head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(5, activation='softmax')
])

#### 4.1.5 Train the model

In [ ]:
## train model
model.compile(
    optimizer=Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_c5 = model.fit(
    ds_train_c5,
    validation_data=ds_val_c5,
    epochs=10
)

#### 4.1.6 Visualizing the loss and accuracy

In [ ]:
import matplotlib.pyplot as plt

# # Loss
# plt.plot(history_c5.history['loss'], label='train_loss')
# plt.plot(history_c5.history['val_loss'], label='val_loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# # Accuracy
# plt.plot(history_c5.history['accuracy'], label='train_accuracy')
# plt.plot(history_c5.history['val_accuracy'], label='val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

### 4.2 ResNet **with finetuning**

#### 4.2.1 Create the model

In [ ]:
# Unfreeze the model
base_model.trainable = True

# Freeze the first 140 layers "freeze"
for layer in base_model.layers[:140]:
    layer.trainable = False

## attach head
model_finetuned = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(5, activation='softmax')
])

#### 4.2.2 Train the model

In [ ]:
from tensorflow.keras.optimizers import Adam

model_finetuned.compile(
    optimizer=Adam(1e-5),    
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_c5_finetune = model_finetuned.fit(
    ds_train_c5,
    validation_data=ds_val_c5,
    epochs=10
)

#### 4.2.3 Visualizing the loss and accuracy after fine tuning

In [ ]:
# Loss
plt.plot(history_c5_finetune.history['loss'], label='train_loss')
plt.plot(history_c5_finetune.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Accuracy
plt.plot(history_c5_finetune.history['accuracy'], label='train_accuracy')
plt.plot(history_c5_finetune.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

***

## 5. Pre-trained Vision Transformer

### 5.1 ViT **without finetuning**

#### 5.1.1 Preprocessing data

In [ ]:
# 1. Setup Generators with 'channels_first'
# We add data_format='channels_first' to match the Hugging Face ViT requirements
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    zoom_range=0.1,
    data_format='channels_first'
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    data_format='channels_first'
)


# 2. Flow from DataFrame
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

val_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,  
    data_format='channels_first'
)

print("Generators recreated with Channels First format.")

# 3. Now run the training code again

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

#### 5.1.2 Model Construction

In [ ]:
# 1. Ensure we know the number of classes (should be 5)
num_classes = len(train_generator.class_indices)

# 2. Load the ViT Base Model (86M Parameters)
model_id = "google/vit-base-patch16-224"

model = TFAutoModelForImageClassification.from_pretrained(
    model_id, 
    num_labels=num_classes, 
    ignore_mismatched_sizes=True
)

# 3. Define Optimizer and Loss (Using tf_keras to avoid version errors)
# ViT requires a small learning rate (5e-5)
optimizer = Adam(learning_rate=5e-5)
loss = CategoricalCrossentropy(from_logits=True)

# 4. Compile the model
# jit_compile=True will help speed up this heavy model on the GPU
model.compile(
    optimizer=optimizer, 
    loss=loss, 
    metrics=['accuracy'],
    jit_compile=True 
)

print(f"Successfully loaded and compiled {model_id}")

#### 5.1.3 Train

In [ ]:
history_ViT = model.fit(
    train_generator,
    epochs=10, 
    validation_data=val_generator,
    callbacks=[early_stopping]
)

### 5.2 ViT **with finetuning**

#### 5.2.1 Model Construction

In [ ]:
# 1. Load the Backbone 
backbone = TFViTModel.from_pretrained("google/vit-base-patch16-224")
backbone.trainable = True 

# 2. Create Input (Channels First)
inputs = Input(shape=(3, 224, 224), name="input_image")

# 3. Backbone Inference
x = backbone(inputs).last_hidden_state
x = x[:, 0, :] # Extract CLS Token

# 4. Custom Hidden Layers
x = layers.Dense(512, activation='relu', name="hidden_layer_1")(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(256, activation='relu', name="hidden_layer_2")(x)
x = layers.Dropout(0.2)(x)

# 5. Output Layer
outputs = layers.Dense(num_classes, name="prediction_head")(x)

# 6. Define the Model
fine_tuned_vit = Model(inputs=inputs, outputs=outputs, name="ViT_With_Custom_Head")

# 7. Compile
optimizer = Adam(learning_rate=1e-4)
loss = CategoricalCrossentropy(from_logits=True)

fine_tuned_vit.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Verify the name in the summary
fine_tuned_vit.summary()

In [ ]:
history_custom = fine_tuned_model.fit(
    train_generator,
    epochs=10, 
    validation_data=val_generator,
    callbacks=[early_stopping]
)

***

## 6. Evaluation

### 6.? ViT

In [ ]:
# 1. Generate Predictions
print("Generating predictions...")
# 'model' is the variable name used in your model.fit() call above
raw_predictions = model.predict(test_generator)

# Check if the model output is a Hugging Face object (TFSequenceClassifierOutput) or a standard Keras Tensor
# TFAutoModelForImageClassification usually returns an object where .logits contains the data
if hasattr(raw_predictions, 'logits'):
    logits = raw_predictions.logits
else:
    logits = raw_predictions

# Convert probabilities/logits to class indices (e.g., [0, 2, 4, 1...])
y_pred = np.argmax(logits, axis=1)

# Get the true labels from the generator
y_true = test_generator.classes

# Get Class Names (e.g., 'Glioma', 'Healthy', etc.)
# We sort the keys to ensure they match the index order (0, 1, 2...)
class_labels = list(test_generator.class_indices.keys())

# 2. Create the Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# 3. Plot the Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, 
            yticklabels=class_labels)

plt.title('Confusion Matrix', fontsize=15)
plt.ylabel('Actual Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45)
plt.show()

# 4. Print Metrics
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

In [ ]:
# code for comparing all models